In [1]:
import numpy as np
import xmlTranslator as xmlT
from tqdm import tqdm
%reload_ext autoreload
%autoreload 2
import importlib
from utils import get_project_root
from ipywidgets import FloatProgress

TODO: 
- Repeat the sequences 3 times
- Extend all chords into 18 enharmonic
- create the new MIDI array 
- Split into train and test
- Pass the MIDI into Embeddings

In [55]:
directory = '/data/iRealXML'
root = get_project_root()
myPath = str(root) + str(directory)

theChordDataset, theOffsetDataset, all_meta = xmlT.parse_info_from_XML(myPath)
print(theChordDataset.shape, theOffsetDataset.shape, len(all_meta))

  0%|          | 0/4007 [00:00<?, ?it/s]

(4006,) (4006,)
(4006,) (4006,) 4006


In [64]:
#find add 4 subtract 3 add 4 subtract 3
for song in theChordDataset:
    for i, e in enumerate(song):
        if 'add 4 subtract 3 add 9' in e:
            print(song)
            print(i)
            break

['<style>' 'Samba' '|:' '.' 'A' '9sus4' 'add 4 subtract 3 add 9' '.' 'A'
 'm69' 'add 4 subtract 3 add 9' '|' '.' 'D' 'm6' '.' 'E' '7' 'add b9' '|'
 '.' 'A' '9sus4' 'add 4 subtract 3 add 9' '.' 'A' 'm69'
 'add 4 subtract 3 add 9' '|' '.' 'D' 'm6' '.' 'E' '7' 'add b9' ':|' '|:'
 '.' 'A' '9sus4' 'add 4 subtract 3' '|' '.' 'G' '7sus4' 'add 7' '.' 'A'
 '7' '/' 'G' '|' '.' 'C' '6' '|' '.' 'B' 'm7' 'alter b5 add b9' '.' 'E'
 '7' 'alter b5 add b9' ':|' '|' '.' 'A' '9sus4' 'add 4 subtract 3 add 9'
 '.' 'A' 'm69' 'add 4 subtract 3 add 9' '|' '.' 'D' 'm6' '.' 'E' '7'
 'add b9' '|' '.' 'A' '9sus4' 'add 4 subtract 3 add 9' '.' 'A' 'm69'
 'add 4 subtract 3 add 9' '|' '.' 'D' 'm6' '.' 'E' '7' 'add b9' '|' '.'
 'A' '9sus4' 'add 4 subtract 3' '|' '.' 'G' '7sus4' 'add 7' '.' 'A' '7'
 '/' 'G' '|' '.' 'C' '69' 'add 9' '|' '.' 'B' 'm7' 'alter b5 add b9' '.'
 'E' '7' 'alter b5 add b9' '|' '.' 'A' '7sus4' 'add 7' '.' 'A' '7' '|:'
 '.' 'D' 'm7' '.' 'B' 'm7' 'alter b5' '.' 'E' '7' '|' '.' 'A' '9sus4'
 'add 4 s

In [65]:
importlib.reload(xmlT)

id = 0
expand_form_dataset = []
status = True
for i in tqdm(range(theChordDataset.shape[0])):
    song_structure, status = xmlT.expand_song_structure(theChordDataset[i], i)
    if status == False:
        print('Song:', all_meta[i])
        id = i
        break
    song_structure = np.array(song_structure)
    expand_form_dataset.append(song_structure)


  0%|          | 0/4006 [00:00<?, ?it/s]

100%|██████████| 4006/4006 [00:00<00:00, 4266.92it/s]


In [66]:
#check the length of the song

max = {'v': 0, 'id': 0}
min = {'v': 1000, 'id': 0}

for i in range(len(expand_form_dataset)):
    if len(expand_form_dataset[i]) > max['v']:
        max['v'] = len(expand_form_dataset[i])
        max['id'] = i
    if len(expand_form_dataset[i]) < min['v']:
        min['v'] = len(expand_form_dataset[i])
        min['id'] = i

print('Max:', max, 'Min:', min)


Max: {'v': 1321, 'id': 141} Min: {'v': 30, 'id': 492}


In [67]:
importlib.reload(xmlT)

#correct the wrong chords
corrected_sequence = xmlT.repleaceTheseChords(expand_form_dataset, False)

  0%|          | 0/4006 [00:00<?, ?it/s]

In [68]:
#Repeat the sequences 3 times
concatenated_array = []

concatenated_array = np.hstack(corrected_sequence) 

vocab = set(concatenated_array)

print(len(vocab))
print(vocab)

288
{'Mambo', 'Gary Aprile', 'Afro-Samba', 'add b9 alter b5', 'add 9 alter #5', 'dom7 add 77', 'm', "Even 8's", 'Slow Swing', 'Form_C', 'add #7 add 9', 'aug', 'add #9 alter b5 add b9', 'Calypso', 'alter b5 add 7', 'add 9 alter b5', 'A', 'Blues Shuffle', 'Cuban Son', 'alter b5', 'Electric Blues', 'alter b5 add #9 add b9', 'Latin', 'Ballad', 'Soul', 'Slow Rock', 'add b6 alter b5', 'N.C.', 'Samba-Rock', 'alter b9 alter #5', 'Blues Rock', 'Samba Reggae', 'alter b5 add b9 alter #5', 'Repeat_0', 'add #9 alter #5', 'Samba-Funk', 'Deliberately', 'add 9 add 7', 'add b9 add #11', 'sus7 add b9', 'Electro Pop', 'Form_B', 'Funk', 'power', 'Pop-Shuffle', 'Slowly', 'alter #5 add #9', 'E', 'add #9 alter b5', "Rock'n'Roll", 'o', 'add #7 alter #5', 'Chacarera', 'add 9 add b13', 'Up Tempo Swing', 'Maxixe', 'Afro', 'Reggae Pop', 'add #11 alter b5', 'Rock Slow', 'UP Swing', 'Ab', 'Baião', 'Country Ballad', 'add #9', 'add 7 alter b9', 'add b9 alter #5', '7 add b9', '|', 'Repeat_1', 'alter b5 add 9', 'add b6

In [69]:
#Add the maj token to the chords
import voicing as vc
voicing = vc.Voicing()
#add the maj token to the chords
new_sequence = None
new_sequence = voicing.add_maj_token(corrected_sequence)

print(new_sequence[3751])

#split any add or alter into a new element in the sequence, the value next to add or alter need to be preserved.



['<style>', 'Funk', '|', 'Form_intro', '.', 'Eb', 'maj7', '/', 'F', '|', '.', 'Bb', 'maj7', '/', 'F', '|', '.', 'C', 'm7', '/', 'F', '|', '.', 'Bb', 'maj7', '/', 'F', 'Form_intro', '.', 'Eb', 'maj7', '/', 'F', '|', '.', 'Bb', 'maj7', '/', 'F', '|', '.', 'C', 'm7', '/', 'F', '|', '.', 'D', 'm7', '.', 'G', 'dom7 add 9', '|', 'Form_A', '.', 'C', 'm7 add 9', '|', '.', 'F', 'sus7 add 9', '.', 'Eb', 'sus7 add 9', '|', '.', 'Bb', 'maj7', '/', 'D', '|', '.', 'G', 'sus7 add 9', '.', 'G', 'dom7 add 9', '|', '.', 'C', 'm7 add 9', '|', '.', 'F', 'sus7 add 9', '.', 'F', 'dom7 add 9', '|', '.', 'Db', 'maj', '/', 'C', '|', '.', 'Bb', 'sus7', 'add 7', '.', 'Bb', 'm', '.', 'Bb', 'dom7', '|', '.', 'Eb', 'm7 add 9', '|', '.', 'Ab', 'sus7 add 9', '.', 'Ab', 'm7 add 9', '|', '.', 'Db', 'maj7', '/', 'A', '|', '.', 'G', 'sus7 add 9', '.', 'G', 'dom7 add 9', '|', '.', 'C', 'sus7 add 9', '.', 'C', 'dom7 add 9', '|', '.', 'F', 'sus7 add 9', '.', 'F', 'dom7', '|', '.', 'Bb', 'maj7', '|', '.', 'G', 'sus7 add 9', 

In [70]:
#fix the format of extensions to avoid repeated elements
natures = {'maj', 'm', 'm6', 'm7', 'dom_7', 'maj7', 'o7', 'o', 'sus', 'sus2', 'sus7', 'ø7', 'power', 'm_maj7'}
id = 3751

ref = 0
for song in tqdm(new_sequence):
    for i, element in enumerate(song):
        if 'add' in element or 'alter' in element:
            #print(element)
            split = element.split(' ')
            size = len(split)
            if size > 2:
                if size%2 == 1:
                    #print('odd', split)
                    nature = split[0]
                    song[i] = nature
                    counter = 1 
                    for n in range(1, size-1, 2):
                        ext = split[n] + ' ' + split[n+1]
                        song.insert(i+counter, ext)
                        counter += 1
    #need to do it separately to avoid the index out of range           
    for i, element in enumerate(song):
        if 'add' in element or 'alter' in element:
            #print(element)
            split = element.split(' ')
            size = len(split)
            if size > 2:   
                if size%2 == 0:
                    #delete the element
                    #print('even:', element, ref, i)
                    song.pop(i)
                    counter = 0
                    for n in range(0, size-1, 2):
                        ext = split[n] + ' ' + split[n+1]
                        song.insert(i+counter, ext)
                        counter += 1
    ref += 1
                    
print(new_sequence[id])

  0%|          | 0/4006 [00:00<?, ?it/s]

100%|██████████| 4006/4006 [00:00<00:00, 10564.59it/s]

['<style>', 'Funk', '|', 'Form_intro', '.', 'Eb', 'maj7', '/', 'F', '|', '.', 'Bb', 'maj7', '/', 'F', '|', '.', 'C', 'm7', '/', 'F', '|', '.', 'Bb', 'maj7', '/', 'F', 'Form_intro', '.', 'Eb', 'maj7', '/', 'F', '|', '.', 'Bb', 'maj7', '/', 'F', '|', '.', 'C', 'm7', '/', 'F', '|', '.', 'D', 'm7', '.', 'G', 'dom7', 'add 9', '|', 'Form_A', '.', 'C', 'm7', 'add 9', '|', '.', 'F', 'sus7', 'add 9', '.', 'Eb', 'sus7', 'add 9', '|', '.', 'Bb', 'maj7', '/', 'D', '|', '.', 'G', 'sus7', 'add 9', '.', 'G', 'dom7', 'add 9', '|', '.', 'C', 'm7', 'add 9', '|', '.', 'F', 'sus7', 'add 9', '.', 'F', 'dom7', 'add 9', '|', '.', 'Db', 'maj', '/', 'C', '|', '.', 'Bb', 'sus7', 'add 7', '.', 'Bb', 'm', '.', 'Bb', 'dom7', '|', '.', 'Eb', 'm7', 'add 9', '|', '.', 'Ab', 'sus7', 'add 9', '.', 'Ab', 'm7', 'add 9', '|', '.', 'Db', 'maj7', '/', 'A', '|', '.', 'G', 'sus7', 'add 9', '.', 'G', 'dom7', 'add 9', '|', '.', 'C', 'sus7', 'add 9', '.', 'C', 'dom7', 'add 9', '|', '.', 'F', 'sus7', 'add 9', '.', 'F', 'dom7', '|

In [43]:
#find 
id = 0
find = ""

for i in range(len(new_sequence)):
    if find in new_sequence[i]:
        print(i)
        print(new_sequence[i])
        id = i
        #break
    
print(all_meta[id])


{'composer': 'George Gershwin, Ira Gershwin (1927)', 'style': 'Medium Swing', 'song_name': "'S Wonderful", 'tonality': 'Eb major', 'midi_key': 15, 'time_signature': '4/4', 'decade': 'Null'}


In [87]:

#find duplicated add or alter elements and remove them

sub_group = new_sequence[0:100]
structural_elements = {'.', '|', ':|', '|:', '/', 'N.C.'}

for x, song in enumerate(new_sequence):
    for y, e in enumerate(song):
        check_extensions = []
        if '.' in e:
            search = True
            stepper = 2
            while search:
                if (stepper + y) >= len(song):
                    search = False
                    #print('End of song')
                    break
                elif y+stepper < len(song) - 1:
                    element = song[y+stepper]
        
                if element in structural_elements or element.startswith('Form_'):
                    search = False
                else:
                    #if element not in check_extensions:
                    check_extensions.append(element)
                    stepper += 1
                    
                        
                        
            if len(check_extensions) > 2:
                #continue
                print(x, y+stepper, check_extensions)
   
print(new_sequence[3729][170:185])
                

12 38 ['dom7', 'add b9', 'alter #5']
12 117 ['dom7', 'add b9', 'alter #5']
12 261 ['dom7', 'add b9', 'alter #5']
13 22 ['m7', 'alter b5', 'add b9']
13 27 ['dom7', 'alter b5', 'add b9']
13 62 ['m7', 'alter b5', 'add b9']
13 67 ['dom7', 'alter b5', 'add b9']
13 138 ['m7', 'alter b5', 'add b9']
13 143 ['dom7', 'alter b5', 'add b9']
16 123 ['m7', 'alter b5', 'add b9']
16 128 ['dom7', 'alter b5', 'add b9']
22 143 ['m7', 'alter b5', 'add b9']
22 148 ['dom7', 'alter b5', 'add b9']
22 262 ['m7', 'alter b5', 'add b9']
22 267 ['dom7', 'alter b5', 'add b9']
23 9 ['maj', 'add 6', 'add 9', 'add 9']
23 16 ['maj', 'add 6', 'add 9', 'add 9']
23 38 ['maj', 'add 6', 'add 9', 'add 9']
23 45 ['maj', 'add 6', 'add 9', 'add 9']
23 50 ['maj', 'add 6', 'add 9']
23 72 ['m7', 'alter b5', 'add b9']
23 77 ['dom7', 'alter b5', 'add b9']
24 143 ['m7', 'alter b5', 'add b9']
24 148 ['dom7', 'alter b5', 'add b9']
24 262 ['m7', 'alter b5', 'add b9']
24 267 ['dom7', 'alter b5', 'add b9']
25 124 ['m7', 'alter b5', 'add b

In [72]:
id = 3998
for i in range(len(theChordDataset[id])):
    print(theChordDataset[id][i], '\t->', new_sequence[id][i])


<style> 	-> <style>
Samba 	-> Samba
|: 	-> |
. 	-> .
A 	-> A
9sus4 	-> sus7
add 4 subtract 3 add 9 	-> add 9
. 	-> .
A 	-> A
m69 	-> m
add 4 subtract 3 add 9 	-> add 6
| 	-> add 9
. 	-> |
D 	-> .
m6 	-> D
. 	-> m6
E 	-> .
7 	-> E
add b9 	-> dom7
| 	-> add b9
. 	-> |
A 	-> .
9sus4 	-> A
add 4 subtract 3 add 9 	-> sus7
. 	-> add 9
A 	-> .
m69 	-> A
add 4 subtract 3 add 9 	-> m
| 	-> add 6
. 	-> add 9
D 	-> |
m6 	-> .
. 	-> D
E 	-> m6
7 	-> .
add b9 	-> E
:| 	-> dom7
|: 	-> add b9
. 	-> .
A 	-> A
9sus4 	-> sus7
add 4 subtract 3 	-> add 9
| 	-> .
. 	-> A
G 	-> m
7sus4 	-> add 6
add 7 	-> add 9
. 	-> |
A 	-> .
7 	-> D
/ 	-> m6
G 	-> .
| 	-> E
. 	-> dom7
C 	-> add b9
6 	-> |
| 	-> .
. 	-> A
B 	-> sus7
m7 	-> add 9
alter b5 add b9 	-> .
. 	-> A
E 	-> m
7 	-> add 6
alter b5 add b9 	-> add 9
:| 	-> |
| 	-> .
. 	-> D
A 	-> m6
9sus4 	-> .
add 4 subtract 3 add 9 	-> E
. 	-> dom7
A 	-> add b9
m69 	-> .
add 4 subtract 3 add 9 	-> A
| 	-> sus7
. 	-> add 9
D 	-> .
m6 	-> A
. 	-> m
E 	-> add 6
7 	-> ad